# Convolutional Neural Network

### Importing the libraries

In [29]:
import tensorflow as tf
import imghdr
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image
import os
import pandas as pd

## Data Preprocessing

### Preprocessing the Training set

In [30]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range = 270,
                                   vertical_flip = True)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [31]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Building the CNN

In [32]:
def cnn_model():
    cnn = tf.keras.models.Sequential() #Initialising the CNN
    cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3])) #Convolution
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) #Pooling
    cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')) #Adding a second convolutional layer
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)) #Second Layer Pooling
    cnn.add(tf.keras.layers.Flatten()) #Flattening
    cnn.add(tf.keras.layers.Dense(units=128, activation='relu')) #Full Connection
    cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) #Output Layer
    cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) #Compiling the CNN
    return cnn

## Training the CNN

### Training the CNN on the Training set and evaluating it on the Test set

In [33]:
cnn_results = cnn_model()
cnn_results.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 50s 199ms/step - loss: 0.6984 - accuracy: 0.5249 - val_loss: 0.6721 - val_accuracy: 0.6060
Epoch 2/25
250/250 [==============================] - 37s 147ms/step - loss: 0.6552 - accuracy: 0.6188 - val_loss: 0.6423 - val_accuracy: 0.6365
Epoch 3/25
250/250 [==============================] - 41s 165ms/step - loss: 0.6324 - accuracy: 0.6418 - val_loss: 0.6572 - val_accuracy: 0.6530
Epoch 4/25
250/250 [==============================] - 43s 171ms/step - loss: 0.6087 - accuracy: 0.6669 - val_loss: 0.5798 - val_accuracy: 0.7025
Epoch 5/25
250/250 [==============================] - 44s 174ms/step - loss: 0.6138 - accuracy: 0.6627 - val_loss: 0.5937 - val_accuracy: 0.6790
Epoch 6/25
250/250 [==============================] - 43s 174ms/step - loss: 0.5951 - accuracy: 0.6769 - val_loss: 0.5983 - val_accuracy: 0.6750
Epoch 7/25
250/250 [==============================] - 43s 174ms/step - loss: 0.5858 - accuracy: 0.6854 - val_loss: 0.5626 - val_ac

In [34]:
cnn_results.predict(test_set)

array([[0.32700673],
       [0.06832469],
       [0.27782148],
       ...,
       [0.9983927 ],
       [0.8219224 ],
       [0.8939076 ]], dtype=float32)

## Making new predictions

In [37]:
def cat_dog_prediction(directory = 'dataset/single_prediction'):
    rows = []
    for f in os.listdir(directory):
        file_type = imghdr.what(directory + '/' + f)
        if file_type in ('png', 'jpeg'):
            test_image = image.load_img(directory + '/' + f, target_size = (64, 64))
            test_image = image.img_to_array(test_image)
            test_image = np.expand_dims(test_image, axis = 0)
            result = cnn_results.predict(test_image)
            training_set.class_indices
            if result[0][0] == 1:
              prediction = 'dog'
            else:
              prediction = 'cat'
        else:
            prediction = 'unsupported_file'
        rows.append([f, prediction])
    results_table = pd.DataFrame(rows, columns=["Image Name", "Prediction"])
    print(results_table)

In [38]:
cat_dog_prediction()

    Image Name        Prediction
0     dog1.jpg               dog
1     cat1.jpg               dog
2     cat2.png               dog
3  2150942.pdf  unsupported_file


In [43]:
testt_datagen = ImageDataGenerator(rescale = 1./255)
testt_set = testt_datagen.image.load_img(directory + '/' + f, target_size = (64, 64))

AttributeError: 'ImageDataGenerator' object has no attribute 'image'